In [1]:
import requests
import pandas as pd
import numpy as np
import time
import math
import os
from dotenv import load_dotenv
import json

In [ ]:
load_dotenv()

api_keys = json.loads(os.getenv("API_KEYS", "[]"))

In [ ]:
BATCH_SIZE = 150   # kamu bisa ubah sesuai kebutuhan


def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(api_keys)

    # hitung berapa batch
    total_batches = math.ceil(total_rows / BATCH_SIZE)
    print(f"Total rows: {total_rows}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Total batches: {total_batches}\n")

    for batch_index in range(total_batches):
        start_row = batch_index * BATCH_SIZE
        end_row = min(start_row + BATCH_SIZE, total_rows)

        print(f"\n=== Processing Batch {batch_index+1}/{total_batches} | Rows {start_row}–{end_row-1} ===\n")

        # copy batch untuk disimpan nanti
        batch_df = df.iloc[start_row:end_row].copy()

        for i in range(start_row, end_row):
            # putar API key berdasarkan row global index
            key_index = i % total_keys
            current_api_key = api_keys[key_index]

            phone_number = df.loc[i, "Whatsapp"]
            result = check_number_status(phone_number, current_api_key)

            batch_df.loc[i, "validity"] = result

            print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | {phone_number} → {result}")

            time.sleep(5)

        # simpan batch ke file
        filename = f"batch_{batch_index+1}.xlsx"
        batch_df.to_excel(filename, index=False)
        print(f"\n💾 Saved: {filename}")

    print("\n🎉 All batches processed successfully!")


python-dotenv could not parse statement starting at line 15


In [2]:
# load data
database = pd.read_excel("cek_ricek.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")

In [ ]:
# no_crm = database[database['CRM'].isna()]
# uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [ ]:
# uncheck['Whatsapp'] = uncheck['Whatsapp'].astype(int)

In [ ]:
# process_dataframe(uncheck)

In [3]:
validity_waiting = pd.read_excel("../../downloads/messages_all_pages.xlsx")
validity_waiting = validity_waiting.drop_duplicates(subset=['phone_number'], keep='last').reset_index(drop=True)

# labeling the validity by error message
validity_waiting['validity'] = np.where(validity_waiting['error'] == "Message Undeliverable.", "Number not registered", "Number registered")

# only get neccesary columns
validity_waiting_fix = validity_waiting[['phone_number', 'validity']]

# rename column
validity_waiting_fix.rename(columns={'phone_number': 'Whatsapp'}, inplace=True)

C:\Users\faruq\AppData\Local\Temp\ipykernel_13940\1454387723.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validity_waiting_fix.rename(columns={'phone_number': 'Whatsapp'}, inplace=True)


In [ ]:
# process_dataframe(uncheck)
# val = pd.read_excel("final_result.xlsx")
# val_1 = pd.read_excel("batch_1.xlsx")
# val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'batch_1.xlsx'

In [ ]:
validity = pd.concat([val_1, val_2])  # , val_2, val_3])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,...,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,Status,validity
0,46530,6282128802917,Zaenuddin,May,2025,NaN,Web Ads,600712,2,PALESTINA,...,05-06,300356,Thu,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
1,48027,628126442337,adelinda pulungan,June,2025,NaN,Web Ads,201297,2,PALESTINA,...,06-07,100648,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
2,52277,6281347808814,Hamba allah,September,2025,NaN,Web Ads,200296,2,PALESTINA,...,19-20,100148,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
3,54955,628121918596,Joe,October,2025,NaN,Web Ads,200789,2,PALESTINA,...,08-09,100394,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
4,55045,628345400400,Andi sudirman,October,2025,NaN,Web Ads,351135,2,PALESTINA,...,14-15,175568,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number not registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,45543,6281289426669,Firdaus,November,2025,NaN,Web Ads,53000,1,PALESTINA,...,07-08,53000,Mon,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Pernah Sukses,Number registered
133,45544,6281229252349,Sugito,November,2025,NaN,Web Ads,100000,1,SUDAN,...,22-23,100000,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Error: 400 Client Error: Bad Request for url: ...
134,45545,6285211979798,Jailani,November,2025,NaN,Web Ads,100714,1,SUDAN,...,07-08,100714,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
135,45547,6285795927968,royani ahmad,November,2025,NaN,Web Ads,100780,1,PALESTINA,...,06-07,100780,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered


In [4]:
validity_update = pd.concat([data_with_validity, validity_waiting_fix])

# # cleansing the number format
validity_update["Whatsapp"] = validity_update["Whatsapp"].astype(str).str.replace(r"\D+", "", regex=True)

# standarize to 62 format
def format_nomor(nomor):
    nomor = str(nomor)
    if nomor.startswith('6'):
        return nomor
    elif nomor.startswith('8'):
        return '62' + nomor
    elif nomor.startswith('0'):
        return '62' + nomor[1:]  
    else:
        return nomor
    
validity_update['Whatsapp'] = validity_update['Whatsapp'].apply(format_nomor)

validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [5]:
duplicates = validity_update[validity_update['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [6]:
validity_update = validity_update[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM', 'Source',
       'Total', 'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori', 'validity']]

In [7]:
validity_waiting_fix['validity'].value_counts(dropna=False)

validity
Number registered        28847
Number not registered     4529
Name: count, dtype: int64

In [8]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")